## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-21-23-51-20 +0000,nyt,Texas Asks Supreme Court to Clear Way for Repu...,https://www.nytimes.com/2025/11/21/us/politics...
1,2025-11-21-23-51-14 +0000,nypost,Trump and Mamdani ‘giddy’ during ‘love fest’ f...,https://nypost.com/2025/11/21/us-news/trump-an...
2,2025-11-21-23-49-59 +0000,cbc,Workers union says it's agreed on 'main points...,https://www.cbc.ca/news/business/canada-post-t...
3,2025-11-21-23-49-57 +0000,bbc,Zelensky warns Ukraine risks losing US support...,https://www.bbc.com/news/articles/cy5g4r9l4dlo...
4,2025-11-21-23-41-51 +0000,cbc,"Ottawa, Alberta expected to sign new energy se...",https://www.cbc.ca/news/politics/alberta-ottaw...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,46
43,new,22
37,peace,17
38,plan,17
33,ukraine,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
98,2025-11-21-20-01-53 +0000,nyt,European Leaders Back Zelensky After Trump’s R...,https://www.nytimes.com/2025/11/21/world/europ...,122
19,2025-11-21-22-54-21 +0000,nyt,"As Trump Pushes on Russia-Ukraine Peace Plan, ...",https://www.nytimes.com/2025/11/21/world/europ...,117
63,2025-11-21-21-33-52 +0000,nypost,Trump says Zohran Mamdani is not a ‘jihadist’ ...,https://nypost.com/2025/11/21/us-news/trump-br...,114
1,2025-11-21-23-51-14 +0000,nypost,Trump and Mamdani ‘giddy’ during ‘love fest’ f...,https://nypost.com/2025/11/21/us-news/trump-an...,114
61,2025-11-21-21-34-00 +0000,wsj,Trump Says He Wants Ukraine’s Answer on Peace ...,https://www.wsj.com/world/zelensky-russia-ukra...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
98,122,2025-11-21-20-01-53 +0000,nyt,European Leaders Back Zelensky After Trump’s R...,https://www.nytimes.com/2025/11/21/world/europ...
1,68,2025-11-21-23-51-14 +0000,nypost,Trump and Mamdani ‘giddy’ during ‘love fest’ f...,https://nypost.com/2025/11/21/us-news/trump-an...
174,68,2025-11-21-15-53-00 +0000,wsj,The Democratic National Committee took out a $...,https://www.wsj.com/politics/policy/democratic...
50,52,2025-11-21-21-55-07 +0000,nypost,Judge was warned not to free violent firebug b...,https://nypost.com/2025/11/21/us-news/judge-wa...
340,49,2025-11-21-01-36-00 +0000,wsj,The Transportation Department said it would pa...,https://www.wsj.com/politics/policy/air-traffi...
23,42,2025-11-21-22-50-21 +0000,nypost,Ex-New York state trooper acquitted of murder ...,https://nypost.com/2025/11/21/us-news/ex-new-y...
20,42,2025-11-21-22-53-44 +0000,nypost,Rep. Eric Swalwell seemingly used DC house at ...,https://nypost.com/2025/11/21/us-news/rep-eric...
183,39,2025-11-21-15-12-08 +0000,nypost,NY Fed president floats chance of a rate cut i...,https://nypost.com/2025/11/21/business/ny-feds...
92,33,2025-11-21-20-17-11 +0000,bbc,Watch: Pope Leo to US students: Don't let AI d...,https://www.bbc.com/news/videos/c99704g0x4ro?a...
46,32,2025-11-21-22-12-56 +0000,nypost,JPMorgan CEO Jamie Dimon hosts birthday bash f...,https://nypost.com/2025/11/21/business/jpmorga...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
